In [1]:
import numpy as np

In [2]:
from keras.models import Model

from keras.layers import Input, Flatten, Dropout, Reshape, Permute, Activation
from keras.layers import Conv2D, Dense, Conv2DTranspose, Cropping2D
from keras.layers import MaxPool2D, Add
from keras.applications import vgg16, vgg19

/Users/user/anaconda3/envs/ml_py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# crop o1 wrt o2
def crop( o1 , o2 , i  ):
    o_shape2 = Model( i  , o2 ).output_shape
    outputHeight2 = o_shape2[1]
    outputWidth2 = o_shape2[2]

    o_shape1 = Model( i  , o1 ).output_shape
    outputHeight1 = o_shape1[1]
    outputWidth1 = o_shape1[2]

    cx = abs( outputWidth1 - outputWidth2 )
    cy = abs( outputHeight2 - outputHeight1 )

    if outputWidth1 > outputWidth2:
        o1 = Cropping2D( cropping=((0,0) ,  (  0 , cx )))(o1)
    else:
        o2 = Cropping2D( cropping=((0,0) ,  (  0 , cx )))(o2)

    if outputHeight1 > outputHeight2 :
        o1 = Cropping2D( cropping=((0,cy) ,  (  0 , 0 )))(o1)
    else:
        o2 = Cropping2D( cropping=((0, cy ) ,  (  0 , 0 )))(o2)

    return o1 , o2 

In [4]:
vgg16_model = vgg16.VGG16(include_top=True, 
                    weights='imagenet', input_tensor=None, 
                    input_shape=None, pooling=None, classes=1000)

In [5]:
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [6]:
layer1 = vgg16_model.get_layer('block1_pool').output
layer2 = vgg16_model.get_layer('block2_pool').output
layer3 = vgg16_model.get_layer('block3_pool').output
layer4 = vgg16_model.get_layer('block4_pool').output
layer5 = vgg16_model.get_layer('block5_pool').output

In [7]:
layers = [layer1,layer2,layer3,layer4,layer5]

In [8]:
def fcn8(num_class,feature_layers,model_input):
    out = feature_layers[-1]
    out = Conv2D(4096, (7,7),
                activation='relu', padding='same')(out)
    out = Dropout(0.5)(out)
    out = Conv2D(4096, (1,1), 
                activation='relu', padding='same')(out)
    out = Dropout(0.5)(out)
    
    out = Conv2D(num_class, (1,1), 
                kernel_initializer='he_normal')(out)
    out = Conv2DTranspose(num_class,
                          kernel_size=(4,4),strides=(2,32),
                         use_bias=False)(out)
    
    out2 = feature_layers[-2]
    out2 = Conv2D(num_class, (1,1), 
                kernel_initializer='he_normal')(out2)
    
    out, out2 = crop(out, out2, model_input)
    
    out = Add()([out, out2])
    
    out = Conv2DTranspose(num_class,
                          kernel_size=(4,4),strides=(2,2),
                         use_bias=False)(out)
    out3 = feature_layers[-3]
    out3 = Conv2D(num_class, (1,1), 
                kernel_initializer='he_normal')(out3)
    
    out3, out = crop(out3, out, model_input)
    out = Add()([out3, out])
    
    out = Conv2DTranspose(num_class,
                          kernel_size=(16,16),strides=(8,8),
                         use_bias=False)(out)
    
    out_shape = Model(model_input, out).output_shape
    
    outHeight = out_shape[1]
    outWidth = out_shape[2]
    
    out = Reshape((-1, outHeight * outWidth))(out)
    out = Permute((2,1))(out)
    out = Activation('softmax')(out)
    model = Model(model_input, out)
    model.outHeight = outHeight
    model.outWidth = outWidth
    
    return model

In [8]:
def fcn32(num_class,feature_layers,model_input):
    out = feature_layers[-1]
    out = Conv2D(4096, (7,7),
                activation='relu', padding='same')(out)
    out = Dropout(0.5)(out)
    out = Conv2D(4096, (1,1), 
                activation='relu', padding='same')(out)
    out = Dropout(0.5)(out)
    
    out = Conv2D(num_class, (1,1), 
                kernel_initializer='he_normal')(out)
    out = Conv2DTranspose(num_class,
                          kernel_size=(64,64),strides=(32,32),
                         use_bias=False)(out)
    out_shape = Model(model_input, out).output_shape
    
    outHeight = out_shape[1]
    outWidth = out_shape[2]
    
    out = Reshape((-1, outHeight * outWidth))(out)
    out = Permute((2,1))(out)
    out = Activation('softmax')(out)
    model = Model(model_input, out)
    model.outHeight = outHeight
    model.outWidth = outWidth
    
    return model

In [9]:
fcn32_model = fcn32(1000, layers, vgg16_model.input)

In [11]:
fcn8_model = fcn8(1000, layers, vgg16_model.input)

In [10]:
fcn32_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
fcn8_model.summary()

NameError: name 'fcn8_model' is not defined

In [14]:
from keras.utils import plot_model

In [15]:
plot_model(fcn8_model, show_shapes=True, to_file='fcn8.png')

In [16]:
plot_model(fcn32_model, show_shapes=True, to_file='fcn32.png')

In [12]:
def reshape_for_fcn(model, input_img):
    predicted_seg = model.predict(np.array([input_img]))[0]
    reshaped_output = predicted_seg.reshape((model.outHeight, model.outWidth, n_classes)).argmax(axis=2)
    
    return reshaped_output

In [13]:
from PIL import Image

In [14]:
cat = Image.open('./cat.jpg')

In [15]:
cat_resized = cat.resize((224,224))
cat_resized_np = np.array(cat_resized)

In [ ]:
seg_cat = reshape_for_fcn(fcn32_model, cat_resized_np)

In [ ]:
seg_cat8 = reshape_for_fcn(fcn8_model, cat_resized_np)

In [ ]:
print(1)